In [1]:
import pandas as pd
import re
import numpy as np
import pickle
from nltk import FreqDist

In [2]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
import nltk
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [5]:
df=pd.read_excel('Data_Train.xlsx')
df.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [6]:
df['SECTION'].value_counts()

1    2772
2    1924
0    1686
3    1246
Name: SECTION, dtype: int64

In [7]:
df=df.drop_duplicates()
df['SECTION'].value_counts()

1    2731
2    1914
0    1673
3    1233
Name: SECTION, dtype: int64

In [8]:
df['STORY'][0]

'But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.\n\n\nGill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.\n\n\nBut there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.'

In [9]:
df['STORY'][10]

'“One would think that their development and testing process would’ve uncovered many of these flaws, and yet they proceeded to put it in the market anyway," said Bryan Ma, vice president of devices research at consultancy IDC. “Clearly they can’t afford to have another embarrassing Note 7-like incident, lest they build up a reputation for releasing unreliable products."\n\n\nThe Note 7 episode triggered a global recall, cost the company billions of dollars and marred its reputation as it battled Apple Inc. in premium devices. Pulling the Fold now lets the Korean giant address potential issues as it races to put out a flexible gadget ahead of Chinese rival Huawei Technologies Co. and Xiaomi Corp.\n\n\nA Samsung spokeswoman declined to comment for this story. Shares in the company, which hasn’t set another date for a commercial launch, were little changed in Seoul on Wednesday.\n\n\nSamsung has bounced back since the Note 7 — it remains the world’s largest producer of smartphones and mem

In [10]:
df_te=pd.read_excel('Data_Test.xlsx')
df_te.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [59]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain','aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [60]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won’t", "will not", phrase)
    phrase = re.sub(r"can’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
   
    return phrase

In [61]:
lemmatizer = WordNetLemmatizer() 
def preprocess_text(sentance):
    sent = decontracted(sentance)

    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\n', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace(',', '')
    sent = re.sub('\sCo\.',' company',sent)
    sent = re.sub('\sCorp\.',' corporation',sent)
    sent = re.sub('&','and',sent)

    sent = re.sub('\$',' dollars ',sent)
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(lemmatizer.lemmatize(e.lower()) for e in sent.split() if e.lower() not in stopwords and len(e)>1)
    #sent = ' '.join(e.lower() for e in sent.split())
    
    return sent

In [62]:
def preprocess_(df,c):
    text=[]  
    for i in tqdm(range(len(df['STORY']))):
        try:
            if df['SECTION'][i]==c:
                text.append(preprocess_text(df['STORY'][i]))      
        except KeyError:
            continue
    return text

In [63]:
X0=preprocess_(df,0)
X1=preprocess_(df,1)
X2=preprocess_(df,2)
X3=preprocess_(df,3)

100%|██████████| 7551/7551 [00:01<00:00, 7203.05it/s]


In [64]:
def words(X):
    w=[]
    for i in X:
        w.extend(i.split())
    return w

In [65]:
w0=words(X0)
w1=words(X1)
w2=words(X2)
w3=words(X3)

In [66]:
freq_dist_w0 = FreqDist(w0)
freq_dist_w1 = FreqDist(w1)
freq_dist_w2 = FreqDist(w2)
freq_dist_w3 = FreqDist(w3)

In [67]:
n=50
print(freq_dist_w0.most_common(n))

[('party', 2001), ('bjp', 1904), ('congress', 1735), ('election', 1592), ('not', 1390), ('said', 1316), ('seat', 1133), ('state', 1119), ('modi', 949), ('minister', 900), ('sabha', 794), ('lok', 793), ('government', 705), ('also', 696), ('leader', 679), ('people', 636), ('india', 632), ('poll', 611), ('political', 583), ('year', 575), ('chief', 524), ('candidate', 524), ('gandhi', 512), ('vote', 508), ('one', 496), ('constituency', 492), ('alliance', 489), ('two', 487), ('time', 466), ('2014', 450), ('phase', 416), ('national', 403), ('delhi', 401), ('pradesh', 399), ('voter', 398), ('new', 380), ('prime', 379), ('issue', 365), ('last', 354), ('janata', 349), ('would', 348), ('assembly', 334), ('first', 331), ('president', 324), ('no', 316), ('country', 306), ('former', 306), ('campaign', 303), ('may', 299), ('narendra', 294)]


In [68]:
print(freq_dist_w1.most_common(n))

[('not', 1313), ('user', 1224), ('also', 1185), ('new', 1183), ('company', 1146), ('said', 1135), ('india', 1128), ('apple', 991), ('phone', 940), ('year', 927), ('smartphone', 914), ('camera', 798), ('device', 795), ('data', 757), ('like', 736), ('google', 717), ('one', 715), ('feature', 688), ('samsung', 675), ('app', 657), ('pro', 643), ('facebook', 614), ('technology', 589), ('market', 572), ('display', 565), ('time', 564), ('would', 534), ('service', 529), ('come', 524), ('galaxy', 500), ('first', 498), ('iphone', 477), ('people', 476), ('million', 456), ('dollar', 450), ('note', 444), ('screen', 439), ('u', 438), ('sale', 435), ('two', 435), ('smartphones', 434), ('huawei', 430), ('according', 425), ('make', 413), ('whatsapp', 396), ('price', 394), ('last', 392), ('platform', 392), ('available', 391), ('use', 385)]


In [69]:
print(freq_dist_w2.most_common(n))

[('film', 1027), ('not', 663), ('also', 433), ('actor', 424), ('said', 386), ('like', 337), ('movie', 335), ('one', 315), ('year', 303), ('time', 286), ('avenger', 273), ('show', 260), ('character', 257), ('would', 242), ('first', 232), ('star', 224), ('endgame', 220), ('story', 209), ('people', 195), ('day', 189), ('director', 183), ('life', 175), ('say', 171), ('man', 164), ('two', 161), ('fan', 158), ('marvel', 157), ('set', 155), ('release', 150), ('new', 149), ('love', 149), ('world', 145), ('make', 143), ('even', 142), ('role', 141), ('work', 135), ('get', 128), ('series', 127), ('back', 126), ('made', 126), ('last', 126), ('way', 125), ('wrote', 123), ('screen', 121), ('know', 120), ('thing', 120), ('audience', 119), ('seen', 119), ('season', 117), ('come', 114)]


In [70]:
print(freq_dist_w3.most_common(n))

[('dollar', 935), ('year', 869), ('said', 802), ('market', 730), ('price', 510), ('company', 458), ('bank', 428), ('india', 419), ('u', 404), ('crore', 384), ('investor', 383), ('growth', 373), ('stock', 363), ('share', 363), ('ltd', 357), ('not', 351), ('month', 329), ('also', 296), ('quarter', 290), ('index', 272), ('currency', 263), ('last', 258), ('oil', 255), ('global', 242), ('march', 242), ('trading', 240), ('trade', 240), ('analyst', 239), ('per', 237), ('rupee', 237), ('new', 237), ('10', 236), ('percent', 230), ('rate', 228), ('point', 228), ('equity', 225), ('china', 224), ('may', 216), ('bond', 215), ('time', 214), ('gold', 213), ('two', 213), ('since', 211), ('indian', 209), ('data', 206), ('week', 201), ('higher', 198), ('high', 196), ('billion', 186), ('yield', 185)]


In [71]:
n=50
s0=[]
for i in freq_dist_w0.most_common(n):
    s0.append(i[0])
s1=[]
for i in freq_dist_w1.most_common(n):
    s1.append(i[0])
s2=[]
for i in freq_dist_w2.most_common(n):
    s2.append(i[0])
s3=[]
for i in freq_dist_w3.most_common(n):
    s3.append(i[0])

In [72]:
c=[]
for i in s1:
    if i in s3:
        c.append(i)
        print(i)

not
also
new
company
said
india
year
data
market
time
dollar
u
two
price
last


In [73]:
c=[]
for i in s0:
    if i in s2:
        c.append(i)
        print(i)

not
said
also
people
year
one
two
time
new
last
would
first


In [12]:
#add words that are common
stopwords= ['i', 'me', 'my', 'myself', 'we','not','said','our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain','aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't", 'also','new','year','one','two','would','may','india']

In [14]:
def preprocess_train(df):
    text=[];label=[];story=[]
    for i in tqdm(range(len(df['SECTION']))):
        try:
            l=re.findall(r'This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed',df['STORY'][i])
            if len(l)!=0 :
                s=df['STORY'][i]
                phrase = re.sub(r"This story has been published from a wire agency feed without modifications to the text. Only the headline has been changed", "",s)
                if len(phrase)>10:
                    story.append(df['STORY'][i])
                    text.append(preprocess_text(phrase))
                    label.append(df['SECTION'][i])
                else:
                    continue
            else:
                story.append(df['STORY'][i])
                text.append(preprocess_text(df['STORY'][i]))
                label.append(df['SECTION'][i])
        except KeyError:
            continue
    return text,label,story

In [15]:
X_p,label_tr,X=preprocess_train(df)
y=np.array(label_tr)
len(X_p),len(y),len(X)

100%|██████████| 7551/7551 [00:09<00:00, 794.10it/s] 


(7469, 7469, 7469)

In [16]:
def preprocess_test(df):
    text=[]  
    for i in tqdm(range(len(df['STORY']))):
         text.append(preprocess_text(df['STORY'][i]))      
    return text

In [17]:
X_te=preprocess_test(df_te)
len(X_te)

100%|██████████| 2748/2748 [00:02<00:00, 1228.35it/s]


2748

In [18]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(X_p)
# we are converting a dictionary with word as a key, and the idf as a value
# dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
# tfidf_words_essay = set(tfidf_model.get_feature_names())

TfidfVectorizer()

In [19]:
X_tr_tfidf=tfidf_model.transform(X_p)
X_tr_tfidf.shape

(7469, 30188)

In [20]:
X_te_tfidf=tfidf_model.transform(X_te)
X_te_tfidf.shape

(2748, 30188)

In [25]:
model_4=MLPClassifier(hidden_layer_sizes=(100,),batch_size=64,early_stopping=True)
model_4.fit(X_tr_tfidf,y)

MLPClassifier(batch_size=64, early_stopping=True)

In [26]:
p=model_4.predict(X_te_tfidf)

In [27]:
pd.DataFrame(p, columns = ['SECTION']).to_excel('predictions_13.xlsx',index=False)

In [28]:
pdf13=pd.read_excel('predictions_13.xlsx')#it got test accuracy of 0.98108
pdf13.head()

,SECTION
0,1
1,2
2,1
3,1
4,1


In [ ]:
# Politics: 0
# Technology: 1
# Entertainment: 2
# Business: 3